# Gold Layer: Streaming Ingestion
## Use Case: User Activity per Event Type per Day
### Aggregate:
count of events per event_type AND grouped by day

### Inputs/Outputs:
Input Silver Table: /mnt/s3mock/silver/events
Output Gold Table: /mnt/s3mock/gold/aggregates

### 1. Init Spark and S3 paths

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, approx_count_distinct

spark = (
    SparkSession.builder.appName("GoldAggregation")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

# S3 Paths
silver_path = "/mnt/s3mock/silver/realtime/events"
gold_path = "/mnt/s3mock/gold/realtime/aggregates"
checkpoint_path = "/mnt/s3mock/checkpoints/gold_aggregates"


### 2. Core logic

In [2]:
silver_df = spark.readStream.format("delta").load(silver_path)

agg_df = silver_df \
    .withColumn("event_date", to_date(col("event_timestamp"))) \
    .groupBy("event_date", "event_type") \
    .agg(
        approx_count_distinct("user_id").alias("unique_users"),
        approx_count_distinct("timestamp").alias("event_count")
    )

query = agg_df.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", checkpoint_path) \
    .start(gold_path)

query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = e128a26b-af85-460f-ab6a-14d3b8132816, runId = e4f14841-f3bb-4d30-9ff5-e79a00f4f34c] terminated with exception: Job aborted due to stage failure: Task 24 in stage 5169.0 failed 1 times, most recent failure: Lost task 24.0 in stage 5169.0 (TID 193391) (91abc9f4dbc0 executor driver): java.lang.IllegalStateException: Error committing version 593 into HDFSStateStore[id=(op=0,part=24),dir=file:/mnt/s3mock/checkpoints/gold_aggregates/state/0/24]
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider$HDFSBackedStateStore.commit(HDFSBackedStateStoreProvider.scala:148)
	at org.apache.spark.sql.execution.streaming.state.StreamingAggregationStateManagerBaseImpl.commit(StreamingAggregationStateManager.scala:89)
	at org.apache.spark.sql.execution.streaming.StateStoreSaveExec.$anonfun$doExecute$7(statefulOperators.scala:436)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util